In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import torch
from transformers import BertForQuestionAnswering, BertTokenizer

class DiabetesChatbot:
    def _init_(self):
        self.model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
        self.tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
        self.max_token_limit = 512
        self.context = """
        Diabetes is a chronic condition that affects how your body processes blood sugar (glucose).
        There are three main types: type 1, type 2, and gestational diabetes.

        Type 1 diabetes occurs when the body doesn't produce enough insulin.
        Type 2 diabetes is the most common type, where the body either doesn't use insulin effectively or doesn't produce enough.
        Gestational diabetes develops during pregnancy but usually goes away after childbirth.

        Common symptoms of diabetes include increased thirst, frequent urination, unexplained weight loss, fatigue, blurred vision, and slow wound healing.

        Treatment options vary depending on the type and severity of diabetes. They may include:
        * Medication (oral medications or injectable insulin)
        * Lifestyle changes (diet, exercise, weight management)
        * Blood sugar monitoring

        Diet: Aim for a balanced diet with plenty of fruits, vegetables, and whole grains. Limit processed foods, sugary drinks, and unhealthy fats.
        Exercise: Regular physical activity helps manage blood sugar levels and improve overall health.

        Blood Sugar Monitoring: Regularly checking your blood sugar levels helps you understand how your body responds to food, activity, and medication.

        Risk Factors: Some common risk factors for developing diabetes include family history, obesity, physical inactivity, and poor diet.

        Complications: Diabetes can lead to various complications such as heart disease, stroke, kidney disease, nerve damage, foot problems, and eye problems.

        Prevention Measures: Maintaining a healthy weight, eating a balanced diet, exercising regularly, avoiding smoking, and limiting alcohol consumption can help prevent or delay the onset of diabetes.

        Precautions During Pregnancy: Pregnant women with diabetes need to carefully monitor their blood sugar levels and follow a specialized meal plan to ensure a healthy pregnancy and baby.

        Lifestyle Changes: Adopting healthy lifestyle habits can significantly improve diabetes management and reduce the risk of complications.

        Long-Term Effects: Untreated or poorly managed diabetes can lead to serious long-term effects on various organs and systems in the body.

        Diabetes in Children: Type 1 diabetes is the most common type of diabetes in children, although type 2 diabetes is becoming increasingly prevalent due to rising obesity rates.

        Attributes:
        * Glucose Level: Normal fasting blood sugar levels typically range from 70 to 100 mg/dL. Levels above 126 mg/dL may indicate diabetes.
        * Blood Pressure: Normal blood pressure is below 120/80 mm Hg. High blood pressure is a risk factor for diabetes complications.
        * Cholesterol and Triglycerides: Healthy cholesterol levels include total cholesterol below 200 mg/dL, LDL (bad) cholesterol below 100 mg/dL, HDL (good) cholesterol above 40 mg/dL for men and above 50 mg/dL for women, and triglycerides below 150 mg/dL.
        * HbA1c (Glycated Hemoglobin): This blood test measures average blood sugar levels over the past 2-3 months. An HbA1c level of 6.5% or higher indicates diabetes.
        * Insulin: Insulin is a hormone that helps regulate blood sugar levels. In diabetes, either the body does not produce enough insulin (type 1) or it becomes resistant to insulin (type 2).
        * BMI (Body Mass Index): BMI is a measure of body fat based on height and weight. A BMI of 18.5 to 24.9 is considered normal, while a BMI of 30 or higher indicates obesity, a risk factor for type 2 diabetes.
        * Diabetes Pedigree Function: This function assesses the probability of diabetes based on family history.

        Research and New Developments: Ongoing research aims to improve diabetes management and develop new treatments, including artificial pancreas systems, gene therapy, and stem cell transplantation.

        Other Important Details: Diabetes management requires a multidisciplinary approach involving healthcare providers, nutritionists, physical therapists, and mental health professionals. Education and support are key components of successful diabetes management.
         Resources:
        * American Diabetes Association: https://www.diabetes.org/
        * National Institute of Diabetes and Digestive and Kidney Diseases: https://www.niddk.nih.gov/health-information/diabetes
        """
        self.external_api_url = "https://your_trusted_diabetes_api.com/api/answer"

    def answer_question(self, question):
        # Split context into chunks to fit within maximum token limit
        context_chunks = [self.context[i:i+self.max_token_limit] for i in range(0, len(self.context), self.max_token_limit)]
        answers = []

        for chunk in context_chunks:
            inputs = self.tokenizer.encode_plus(question, chunk, return_tensors="pt", add_special_tokens=True)

            with torch.no_grad():
                outputs = self.model(**inputs)

            answer_start_scores = outputs.start_logits
            answer_end_scores = outputs.end_logits

            answer_start = torch.argmax(answer_start_scores)
            answer_end = torch.argmax(answer_end_scores) + 1

            answer = self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

            answers.append(answer)

        return " ".join(answers)

def main():
    st.title("Diabetes Prediction and Chatbot")
    st.sidebar.title("Diabetes Management Assistant")

    bot = DiabetesChatbot()

    st.sidebar.write("Hi! How can I assist you with your diabetes management today?")
    user_input = st.sidebar.text_input("You:")

    if user_input:
        if user_input.lower() == "quit":
            st.sidebar.write("Bot: Goodbye!")
        else:
            response = bot.answer_question(user_input)
            st.sidebar.write("Bot:", response)

    st.write("Diabetes Prediction")
    st.write("Enter your details below for prediction:")

    # Load your dataset
    df = pd.read_csv('diabetes.csv')

    # Prepare the data (convert tabular data into sequences)
    # Example: You might want to create sequences based on time intervals

    # Split the data into train and test sets
    x = df.drop(['Outcome'], axis=1)
    y = df['Outcome'].astype('int')
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

    # Model Architecture with Variations
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(0.3, name='dropout_1'))  # Updated name to ensure uniqueness
    model.add(LSTM(units=64, return_sequences=True))
    model.add(Dropout(0.3, name='dropout_2'))  # Updated name to ensure uniqueness
    model.add(LSTM(units=32))
    model.add(Dropout(0.2, name='dropout_3'))  # Updated name to ensure uniqueness
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(x_train.values.reshape(x_train.shape[0], x_train.shape[1], 1), y_train, epochs=15, batch_size=64)

    # Function to get user input
    def user_report():
        pregnancies = st.number_input('Pregnancies', min_value=0, max_value=17, value=3)
        glucose = st.number_input('Glucose', min_value=0, max_value=200, value=120)
        bp = st.number_input('Blood Pressure', min_value=0, max_value=122, value=70)
        skinthickness = st.number_input('Skin Thickness', min_value=0, max_value=100, value=20)
        insulin = st.number_input('Insulin', min_value=0, max_value=846, value=79)
        bmi = st.number_input('BMI', min_value=0, max_value=67, value=20)
        dpf = st.number_input('Diabetes Pedigree Function', min_value=0.0, max_value=2.4, value=0.47)
        age = st.number_input('Age', min_value=21, max_value=88, value=33)

        return pregnancies, glucose, bp, skinthickness, insulin, bmi, dpf, age

    # Get user input
    pregnancies, glucose, bp, skinthickness, insulin, bmi, dpf, age = user_report()
    if st.button('Result'):
        # Prepare user data for prediction
        user_data_reshaped = np.array([[pregnancies, glucose, bp, skinthickness, insulin, bmi, dpf, age]]).reshape(1, 8, 1)

        # Prediction
        user_result = model.predict(user_data_reshaped)

        # Convert probabilities to binary class labels using a threshold (e.g., 0.5)
        user_result_class = 1 if user_result[0][0] >= 0.5 else 0

        # Output
        st.subheader('Your Report:')
        output = 'You are Diabetic' if user_result_class == 1 else 'You are not Diabetic'
        st.title(output)

        # Evaluate model's accuracy and F1 score on test set
        test_predictions = model.predict(x_test.values.reshape(x_test.shape[0], x_test.shape[1], 1))
        test_predictions_classes = (test_predictions >= 0.5).astype(int)
        test_accuracy = accuracy_score(y_test, test_predictions_classes)
        f1 = f1_score(y_test, test_predictions_classes)
        st.subheader('Metrics:')
        st.write("Model's accuracy on test set: " + str(test_accuracy))
        st.write("Model's F1 score on test set: " + str(f1))

if _name_ == "_main_":
    main()